## Run copairs on all evalzoo results and check if AP values match

In [3]:
import sys

sys.path.append("../scripts")

import logging
from pathlib import Path

import pandas as pd
import numpy as np

from load import load_config
from copairs.map import run_pipeline, aggregate

import seaborn as sns
from matplotlib import pyplot as plt

from tqdm.auto import tqdm

import pickle

#### Run copairs on evalzoo configs with existing results

In [4]:
def run_copairs_on_evalzoo(evalzoo_result, null_size=10000, batch_size=100000):
    config = load_config(f"../2.evalzoo/results/{evalzoo_result}/params.yaml")
    profiles_path = config["experiment"]["input_structure"].format(
        **config["experiment"]
    )

    # check if exists
    if Path(profiles_path).exists():
        ann_df = pd.read_parquet(profiles_path)

        ann_df.reset_index(inplace=True, drop=True)
        metadata = ann_df.filter(regex="^Metadata_")
        feature_values = ann_df.filter(regex="^(?!Metadata_)").values
        assert "Metadata_pert_type" in metadata.columns

        # add dummy index to speed up calculations by skipping positive pairs between controls
        metadata.loc[:, "Metadata_dummy_index"] = metadata.index
        metadata.loc[
            metadata["Metadata_pert_type"] == "trt", "Metadata_dummy_index"
        ] = -1

        pos_sameby = config["experiment"]["sim_params"]["all_same_cols_rep"] or []
        pos_sameby = [c for c in pos_sameby if c != "Metadata_reference_or_other"]
        pos_diffby = config["experiment"]["sim_params"]["all_different_cols_rep"] or []
        pos_diffby = [pos_diffby] if isinstance(pos_diffby, str) else pos_diffby

        neg_sameby = []
        neg_diffby = ["Metadata_pert_type"]

        print("Sameby:", pos_sameby, "Diffby:", pos_diffby)

        copairs_result, _, _ = run_pipeline(
            metadata,
            feature_values,
            pos_sameby + ["Metadata_dummy_index"],
            pos_diffby,
            neg_sameby,
            neg_diffby,
            null_size=null_size,
            batch_size=batch_size,
        )

        return aggregate(copairs_result, sameby=pos_sameby, threshold=0.05)
    else:
        logging.warning(f"File {profiles_path} not found. Skipping.")
        return None

In [5]:
copairs_results = {}

In [4]:
results_dir = Path("../2.evalzoo/results/")

for subdir in tqdm(results_dir.iterdir()):
    collatedsim_path = subdir / "collatedsim.parquet"
    l_0_1_path = subdir / "metrics_level_1_0_ref.parquet"

    if collatedsim_path.exists() and l_0_1_path.exists():
        subdir_collatedsim = pd.read_parquet(collatedsim_path)
        subdir_l_1_0 = pd.read_parquet(l_0_1_path)

        # create groupby objects for both DataFrames
        group_col = subdir_l_1_0.columns[1]
        collatedsim_groups = subdir_collatedsim.groupby(group_col)["id1"].unique()
        l_1_0_groups = subdir_l_1_0.groupby(group_col)["id1"].unique()

        # find groups (values of Metadata_JCP2022) that exist in both DataFrames
        common_groups = set(collatedsim_groups.index).intersection(
            set(l_1_0_groups.index)
        )

        # compare the unique id1 values for each common group
        mismatch = any(
            set(collatedsim_groups[jcp]) != set(l_1_0_groups[jcp])
            for jcp in common_groups
        )

        # check if there was a mismatch
        if mismatch:
            print(f"{subdir.name} id1 mismatch")
        else:
            print(f"{subdir.name} id1 match OK")

        copairs_results[subdir.name] = run_copairs_on_evalzoo(subdir.name)

        # pickle intermediate results
        with open("output/copairs_results.pkl", "wb") as f:
            pickle.dump(copairs_results, f)

0it [00:00, ?it/s]

/var/folders/41/1jdfcmfs0gx7qwx9d31rtxgr0000gq/T/ipykernel_85051/2184141838.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata.loc[:, "Metadata_dummy_index"] = metadata.index
2023-07-03 15:34:14 INFO     Indexing metadata...
2023-07-03 15:34:14 INFO     Finding positive pairs...


630757cf id1 match OK
Sameby: ['Metadata_JCP2022'] Diffby: ['Metadata_Well']


2023-07-03 15:34:14 INFO     dropping dups...
2023-07-03 15:34:14 INFO     Finding negative pairs...
2023-07-03 15:34:34 INFO     dropping dups...
2023-07-03 15:35:18 INFO     Computing positive similarities...
2023-07-03 15:35:18.568863: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  0%|          | 0/1 [00:00<?, ?it/s]

...

2023-07-03 18:22:53 INFO     dropping dups...
2023-07-03 18:22:53 INFO     Finding negative pairs...
2023-07-03 18:23:13 INFO     dropping dups...
2023-07-03 18:23:59 INFO     Computing positive similarities...


  0%|          | 0/1 [00:00<?, ?it/s]

2023-07-03 18:24:00 INFO     Computing negative similarities...


  0%|          | 0/161 [00:00<?, ?it/s]

2023-07-03 18:24:34 INFO     Building rank lists...
2023-07-03 18:24:55 INFO     Computing average precision...
2023-07-03 18:24:55 INFO     Computing null distributions...
/Users/akalinin/Projects/copairs/src/copairs/compute_np.py:107: RuntimeWarning: invalid value encountered in divide
  ap = (pr_k * rel_k).sum(axis=1) / num_pos
/Users/akalinin/Projects/copairs/src/copairs/compute_np.py:107: RuntimeWarning: invalid value encountered in divide
  ap = (pr_k * rel_k).sum(axis=1) / num_pos
/Users/akalinin/Projects/copairs/src/copairs/compute_np.py:107: RuntimeWarning: invalid value encountered in divide
  ap = (pr_k * rel_k).sum(axis=1) / num_pos
/Users/akalinin/Projects/copairs/src/copairs/compute_np.py:107: RuntimeWarning: invalid value encountered in divide
  ap = (pr_k * rel_k).sum(axis=1) / num_pos
2023-07-03 18:25:02 INFO     Computing P-values...
2023-07-03 18:25:03 INFO     Creating result DataFrame...
2023-07-03 18:25:03 INFO     Finished.


#### Compare copairs results with evalzoo

In [42]:
with open("output/copairs_results.pkl", "rb") as f:
    copairs_results = pickle.load(f)

In [43]:
copairs_results_nona = {k: v for k, v in copairs_results.items() if v is not None}
len(copairs_results), len(copairs_results_nona)

(64, 60)

In [44]:
mismatches = []
for key in copairs_results_nona.keys():
    copairs_result = copairs_results_nona[key][
        ~copairs_results_nona[key].average_precision.isna()
    ]
    evalzoo_result = pd.read_parquet(
        f"../2.evalzoo/results/{key}/metrics_level_1_ref.parquet"
    )
    evalzoo_result = evalzoo_result.groupby(
        copairs_result.filter(regex="^Metadata_").columns.tolist()
    ).mean(numeric_only=True)

    if np.allclose(
        copairs_result.average_precision,
        evalzoo_result.sim_retrieval_average_precision_ref_i_mean_i,
    ):
        print(f"{key} match OK")
    else:
        print(f"{key} mismatch")
        mismatches.append(key)

630757cf match OK
66fd2a4d match OK
f86c9fcc match OK
ac65bc4b match OK
273d3138 match OK
263a5ff4 match OK
2bde8689 match OK
f5c36cb4 match OK
30f99c08 match OK
125bfb5b match OK
8a46f718 match OK
3d645cb2 match OK
d2311b58 match OK
d4df76cd match OK
26efacc8 match OK
5ee8aabb match OK
64aa0540 match OK
faa6ba3e match OK
49abf604 match OK
790b8557 match OK
fefae721 match OK
cdaf16db match OK
ef524325 match OK
b2e52a3a match OK
82ad004e match OK
df66eaf7 match OK
b3ded838 match OK
703b13da match OK
5a048fe4 match OK
0e4cd68f match OK
bcf32878 match OK
61da8ae0 match OK
b8ad0e58 match OK
21b92180 match OK
d66f1bce match OK
0485b96b match OK
ceb5f02a match OK
cd1b92de match OK
fd50b01b match OK
63526422 match OK
5eaae264 match OK
e2bf955d match OK
758dee96 match OK
889ca0ab match OK
0cc6a4ec match OK
adacbee3 match OK
3701ed2c match OK
df015b9f match OK
edaff3b6 match OK
6fae5285 match OK
304eb132 match OK
c4c7aa12 match OK
34fbd8a4 match OK
10792e71 match OK
1b22d880 match OK
306333dc m

In [45]:
mismatches

[]